In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

/home/cristiandugacicu/.local/lib/python2.7/site-packages/pyspark/context.py:227: DeprecationWarning: Support for Python 2 and Python 3 prior to version 3.6 is deprecated as of Spark 3.0. See also the plan for dropping Python 2 support at https://spark.apache.org/news/plan-for-dropping-python-2-support.html.
  DeprecationWarning)


In [3]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv
!wc -l fhvhv_tripdata_2021-01.csv

11908469 fhvhv_tripdata_2021-01.csv


In [4]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')
df.headdf.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,StringType,true),StructField(DOLocationID,StringType,true),StructField(SR_Flag,StringType,true)))

In [7]:
display(df.head(5))
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

[Row(hvfhs_license_num=u'HV0003', dispatching_base_num=u'B02682', pickup_datetime=u'2021-01-01 00:33:44', dropoff_datetime=u'2021-01-01 00:49:07', PULocationID=u'230', DOLocationID=u'166', SR_Flag=None),
 Row(hvfhs_license_num=u'HV0003', dispatching_base_num=u'B02682', pickup_datetime=u'2021-01-01 00:55:19', dropoff_datetime=u'2021-01-01 01:18:21', PULocationID=u'152', DOLocationID=u'167', SR_Flag=None),
 Row(hvfhs_license_num=u'HV0003', dispatching_base_num=u'B02764', pickup_datetime=u'2021-01-01 00:23:56', dropoff_datetime=u'2021-01-01 00:38:05', PULocationID=u'233', DOLocationID=u'142', SR_Flag=None),
 Row(hvfhs_license_num=u'HV0003', dispatching_base_num=u'B02764', pickup_datetime=u'2021-01-01 00:42:51', dropoff_datetime=u'2021-01-01 00:45:50', PULocationID=u'142', DOLocationID=u'143', SR_Flag=None),
 Row(hvfhs_license_num=u'HV0003', dispatching_base_num=u'B02764', pickup_datetime=u'2021-01-01 00:48:14', dropoff_datetime=u'2021-01-01 01:08:42', PULocationID=u'143', DOLocationID=u'7

In [10]:
import pandas as pd
df_pandas = pd.read_csv('head.csv')
display(df_pandas.dtypes)

# create Spark dataFrame from Panda
spark.createDataFrame(df_pandas).schema

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,DoubleType,true)))

In [47]:
from pyspark.sql import types

schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

display(df.head(5))

df = df.repartition(24)

#split big file in 24 small files:
df.write.parquet('fhvhv/2021/01/', mode='overwrite')

df = spark.read.parquet('fhvhv/2021/01/')

df.printSchema()


[Row(hvfhs_license_num=u'HV0003', dispatching_base_num=u'B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num=u'HV0003', dispatching_base_num=u'B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num=u'HV0003', dispatching_base_num=u'B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142, SR_Flag=None),
 Row(hvfhs_license_num=u'HV0003', dispatching_base_num=u'B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143, SR_Flag=None),
 Row(hvfhs_license_num=u'HV0003', dispatching_base_num=u'B02764', 

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [50]:
def crazy_stuff(num):
  if num % 7 == 0:
    return 's/{}'.format(num)
  elif num % 3 == 0:
    return 'a/{}'.format(num)
  else:
    return 'e/{}'.format(num)

display(crazy_stuff('B02884'))

crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

TypeError: not all arguments converted during string formatting

In [49]:
# Create new colums by applying Spark functions

from pyspark.sql import functions as F

df.withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
  .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
  .withColumn('base_id', crazy_stuff_udf(df.PULocationID)) \
  .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
  .show()


PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/home/cristiandugacicu/.local/lib/python2.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 605, in main
    process()
  File "/home/cristiandugacicu/.local/lib/python2.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 597, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/cristiandugacicu/.local/lib/python2.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 223, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/home/cristiandugacicu/.local/lib/python2.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 141, in dump_stream
    for obj in iterator:
  File "/home/cristiandugacicu/.local/lib/python2.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 212, in _batched
  

In [22]:
# df \
#     .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
#     .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
#     .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
#     .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
#     .show()

df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003') \
  .show()


# !head -n 10 head.csv

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-03 14:02:32|2021-01-03 14:10:21|         126|          47|
|2021-01-03 20:36:10|2021-01-03 20:59:38|         223|         243|
|2021-01-04 07:59:44|2021-01-04 08:06:24|         191|          38|
|2021-01-04 09:36:18|2021-01-04 09:42:16|          18|         235|
|2021-01-04 17:20:40|2021-01-04 17:38:31|          80|         138|
|2021-01-03 04:12:00|2021-01-03 04:28:57|         263|         193|
|2021-01-03 12:04:09|2021-01-03 12:28:20|         258|          62|
|2021-01-02 16:50:38|2021-01-02 17:02:40|         189|          17|
|2021-01-01 09:34:39|2021-01-01 09:58:17|         224|          51|
|2021-01-02 00:14:05|2021-01-02 00:44:08|         132|         230|
|2021-01-04 12:34:21|2021-01-04 12:40:31|         146|         229|
|2021-01-02 01:30:31|2021-01-02 01:38:59|       